In [1]:
from products.models import Allergy, Category, Image, Menu, Nutrition, Product

# 1:N 모델 참조

## 1.1. 정참조

In [3]:
bread = Category.objects.get(name="브레드")
print(bread.menu, bread.menu.name)

Menu object (2) 푸드


## 1.2. 역참조

아래 모델에서 Menu와 Category는 1:N관계이다
Category에서 FK인 menu를 통해서 menu를 접근할 수 있으나, 반대의 경우인 menu에서 category로는 특수한 방법을 사용해야된다.

```python
class Menu(models.Model): 
    name = models.CharField(max_length=45)

    class Meta: 
        db_table = 'menus'

class Category(models.Model): 
    name = models.CharField(max_length=45)
    menu = models.ForeignKey(Menu, on_delete=models.CASCADE, related_name="category")

    class Meta: 
        db_table = 'categories'
```

### 1.2.1. \[classname]_set을 이용해서 참조

In [15]:
food = Menu.objects.get(name="푸드")
print(food.category_set.all())

<QuerySet [<Category: Category object (10)>, <Category: Category object (11)>, <Category: Category object (12)>, <Category: Category object (13)>, <Category: Category object (14)>, <Category: Category object (15)>, <Category: Category object (16)>]>


### 1.2.2. related_name을 이용해서 참조
related_name 으로 접근하는 방법

In [3]:
food = Menu.objects.get(name="푸드")
print(food.category.all())

<QuerySet [<Category: Category object (10)>, <Category: Category object (11)>, <Category: Category object (12)>, <Category: Category object (13)>, <Category: Category object (14)>, <Category: Category object (15)>, <Category: Category object (16)>]>


`related_name` 정의 시 더 이상 `[classname]_set`을 사용할 수 없다

In [2]:
food = Menu.objects.get(name="푸드")
print(food.category_set.all())

AttributeError: 'Menu' object has no attribute 'category_set'

# 2. 1:1 모델 참조 (OneToOneField)

## 1.1. 정참조
1:N 모델과 동일

## 1.2. 역참조

OneToOneField를 사용한 1:1 모델에서는 `[classname]_set`으로 접근하지 않아도 된다, 어짜피 1개이기 때문에, `[classname]`으로 접근하면 된다.

In [12]:
nutrition = Nutrition.objects.get(id=3)
print(nutrition.product, nutrition.product.korean_name)

Product object (2) 콜드 브루


# 3. N:M 모델 참조 (ManyToManyField)

## 3.1. ManyToManyField가 있는 테이블에서 참조

`[ManyToManyField명].through`로 정션테이블 접근 가능

In [15]:
product = Product.objects.get(korean_name="콜드 브루")
product_allergies = product.allergy.through.objects.all()

for product_allergy in product_allergies:
    print(product_allergy.allergy, product_allergy.allergy.name)
    

Allergy object (1) 대두
Allergy object (2) 우유
Allergy object (3) 난류
Allergy object (4) 밀


`[ManyToManyField명]`로 접근 가능

In [29]:
product = Product.objects.get(korean_name="바나나 슈크림 큐브")
allergies = product.allergy.all()

for allergy in allergies:
    print(allergy, allergy.name)
    

Allergy object (1) 대두
Allergy object (2) 우유
Allergy object (3) 난류
Allergy object (4) 밀


## 3.2. ManyToManyField가 없는 테이블에서 참조

`[targetTable]_set.through`로 정션테이블 접근 가능

In [30]:
allergy = Allergy.objects.get(name="대두")
product_allergies = allergy.product_set.through.objects.all()

for product_allergy in product_allergies:
    print(product_allergy.allergy, product_allergy.allergy.name)
    

Allergy object (1) 대두
Allergy object (2) 우유
Allergy object (3) 난류
Allergy object (4) 밀
